In [2]:
import geopandas as gpd
import pandas as pd
import folium
import matplotlib.cm as cm
import matplotlib.colors as mcolors
from geopy.geocoders import Nominatim
from tqdm import tqdm
from time import sleep

In [3]:
import osmnx as ox
print(ox.__version__)
dir(ox)  # Look for 'geometries_from_place'

2.0.4


['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 '_errors',
 '_http',
 '_nominatim',
 '_osm_xml',
 '_overpass',
 'add_edge_bearings',
 'add_edge_grades',
 'add_edge_speeds',
 'add_edge_travel_times',
 'add_node_elevations_google',
 'add_node_elevations_raster',
 'basic_stats',
 'bearing',
 'citation',
 'consolidate_intersections',
 'convert',
 'distance',
 'elevation',
 'features',
 'features_from_address',
 'features_from_bbox',
 'features_from_place',
 'features_from_point',
 'features_from_polygon',
 'features_from_xml',
 'geocode',
 'geocode_to_gdf',
 'geocoder',
 'graph',
 'graph_from_address',
 'graph_from_bbox',
 'graph_from_gdfs',
 'graph_from_place',
 'graph_from_point',
 'graph_from_polygon',
 'graph_from_xml',
 'graph_to_gdfs',
 'io',
 'k_shortest_paths',
 'load_graphml',
 'log',
 'metadata_version',
 'nearest_edges',
 'nearest_nodes',
 'orientation_entropy',
 'plot',
 'plot_figure

In [4]:
import osmnx as ox
import geopandas as gpd

# ✅ Use the legacy-compatible function
berlin_bezirke = ox.features_from_place(
    "Berlin, Germany",
    tags={"admin_level": "9", "boundary": "administrative"}
)

# Clean and prepare
berlin_bezirke = berlin_bezirke[["name", "geometry"]].reset_index(drop=True)
berlin_bezirke = berlin_bezirke.rename(columns={"name": "neighborhood"})
berlin_bezirke = berlin_bezirke.to_crs("EPSG:4326")

print(berlin_bezirke["neighborhood"].unique())

['Reinickendorf' 'Frohnau' 'Spandau' 'Charlottenburg-Nord' 'Mitte'
 'Hansaviertel' 'Wedding' 'Moabit' 'Gesundbrunnen' 'Deutschland'
 'Steglitz-Zehlendorf' 'Steglitz' 'Lankwitz' 'Lichterfelde' 'Dahlem'
 'Wilmersdorf' 'Halensee' 'Grunewald' 'Wittenau' 'Märkisches Viertel'
 'Hermsdorf' 'Tegel' 'Heiligensee' 'Tiergarten' 'Schöneberg' 'Friedenau'
 'Marienfelde' 'Treptow-Köpenick' 'Adlershof' 'Rudow' 'Johannisthal'
 'Niederschöneweide' 'Baumschulenweg' 'Plänterwald' 'Alt-Treptow'
 'Friedrichshain' 'Friedrichshain-Kreuzberg' 'Kreuzberg' 'Großbeeren'
 'Blankenfelde-Mahlow' 'Schönefeld' 'Eichwalde' 'Schulzendorf' 'Zeuthen'
 'Königs Wusterhausen' 'Gosen-Neu Zittau' 'Spreenhagen' 'Erkner'
 'Konradshöhe' 'Potsdam-Mittelmark' 'Teltow-Fläming' 'Potsdam'
 'Dahme-Spreewald' 'Havelland' 'Berlin' 'Oberhavel' 'Brandenburg' 'Barnim'
 'Oder-Spree' 'Waidmannslust' 'Charlottenburg' 'Grünau' 'Waltersdorf'
 'Schmöckwitz' 'Gosen' 'Wernsdorf' 'Lichtenrade' 'Tempelhof-Schöneberg'
 'Buckow' 'Tempelhof' 'Mariendorf

### ✅ List of official Bezirke used in our project
valid_bezirke = [
    "Mitte", "Friedrichshain-Kreuzberg", "Pankow", "Charlottenburg-Wilmersdorf",
    "Spandau", "Steglitz-Zehlendorf", "Tempelhof-Schöneberg", "Neukölln",
    "Treptow-Köpenick", "Lichtenberg", "Reinickendorf", "Marzahn-Hellersdorf"
]

In [5]:
marzahn_hellersdorf_neighborhoods = [
    "Biesdorf", "Hellersdorf", "Kaulsdorf", "Marzahn", "Mahlsdorf"]
mitte_neighborhoods = [
    "Mitte", "Wedding", "Tiergarten", "Moabit", "Hansaviert", "Wedding"]
friedrichshain_kreuzberg_neighborhoods = [
    "Friedrichshain", "Kreuzberg", "Kreuzberg-Nord", "Kreuzberg-Süd"]
pankow_neighborhoods = [
    "Pankow", "Prenzlauer Berg", "Weißensee", "Blankenburg", "Heinersdorf"]
charlottenburg_wilmersdorf_neighborhoods = [
    "Charlottenburg", "Wilmersdorf", "Westend", "Grunewald", "Halensee"]
spandau_neighborhoods = [
    "Spandau", "Staaken", "Falkenhagener Feld", "Siemensstadt", "Haselhorst"]
steglitz_zehlendorf_neighborhoods = [
    "Steglitz", "Zehlendorf", "Wannsee", "Nikolassee", "Dahlem"]
tempelhof_schoeneberg_neighborhoods = [
    "Tempelhof", "Schöneberg", "Mariendorf", "Lichtenrade", "Friedenau"]
neukoelln_neighborhoods = [
    "Neukölln", "Britz", "Buckow", "Gropiusstadt", "Rudow"]
treptow_koepenick_neighborhoods = [
    "Treptow", "Köpenick", "Oberschöneweide", "Schöneweide", "Alt-Treptow"]
lichtenberg_neighborhoods = [
    "Lichtenberg", "Falkenberg", "Hohenschönhausen", "Karlshorst", "Rummelsburg"]
reinickendorf_neighborhoods = [
    "Reinickendorf", "Tegel", "Heiligensee", "Frohnau", "Waidmannslust"]    

In [6]:
import folium

# 🗺️ Base map centered over Berlin
m = folium.Map(location=[52.52, 13.405], zoom_start=10, tiles="OpenStreetMap")

# 🎨 Define colors for each Bezirk
bezirk_colors = {
    "Marzahn-Hellersdorf": "#FF6B6B",
    "Mitte": "#4ECDC4", 
    "Friedrichshain-Kreuzberg": "#45B7D1",
    "Pankow": "#96CEB4",
    "Charlottenburg-Wilmersdorf": "#FFEAA7",
    "Spandau": "#DDA0DD",
    "Steglitz-Zehlendorf": "#98D8C8",
    "Tempelhof-Schöneberg": "#F7DC6F",
    "Neukölln": "#BB8FCE",
    "Treptow-Köpenick": "#85C1E9",
    "Lichtenberg": "#F8C471",
    "Reinickendorf": "#82E0AA"
}

# 🎯 Create separate layers for each Bezirk with toggle functionality
bezirk_data = {
    "Marzahn-Hellersdorf": marzahn_hellersdorf_neighborhoods,
    "Mitte": mitte_neighborhoods,
    "Friedrichshain-Kreuzberg": friedrichshain_kreuzberg_neighborhoods,
    "Pankow": pankow_neighborhoods,
    "Charlottenburg-Wilmersdorf": charlottenburg_wilmersdorf_neighborhoods,
    "Spandau": spandau_neighborhoods,
    "Steglitz-Zehlendorf": steglitz_zehlendorf_neighborhoods,
    "Tempelhof-Schöneberg": tempelhof_schoeneberg_neighborhoods,
    "Neukölln": neukoelln_neighborhoods,
    "Treptow-Köpenick": treptow_koepenick_neighborhoods,
    "Lichtenberg": lichtenberg_neighborhoods,
    "Reinickendorf": reinickendorf_neighborhoods
}

# 🗂️ Add each Bezirk as a separate toggleable layer
for bezirk_name, neighborhoods in bezirk_data.items():
    # Filter for this specific Bezirk's neighborhoods
    bezirk_gdf = berlin_bezirke[berlin_bezirke["neighborhood"].isin(neighborhoods)]
    
    if not bezirk_gdf.empty:
        # Create a feature group for this Bezirk (allows toggling)
        feature_group = folium.FeatureGroup(name=f"🏘️ {bezirk_name}")
        
        # Add neighborhoods to this Bezirk's layer
        folium.GeoJson(
            data=bezirk_gdf,
            style_function=lambda x, color=bezirk_colors[bezirk_name]: {
                "fillColor": color,
                "color": "black",
                "weight": 2.0,
                "fillOpacity": 0.6,
                "opacity": 1.0
            },
            tooltip=folium.GeoJsonTooltip(
                fields=["neighborhood"], 
                aliases=[f"{bezirk_name} - Neighborhood:"],
                localize=True
            ),
            popup=folium.GeoJsonPopup(
                fields=["neighborhood"],
                aliases=[f"{bezirk_name}:"]
            )
        ).add_to(feature_group)
        
        # Add this Bezirk layer to the map
        feature_group.add_to(m)

# 🧭 Layer control (allows toggling each Bezirk on/off)
folium.LayerControl(collapsed=False, position='topright').add_to(m)

# Display the map
m